In [1]:
import quandl
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing, cross_validation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh'

/home/carla/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = []
Train = []
for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt']
    time = [2,4,7,30,60,120,250]
    Z = pd.DataFrame()
    for f in func:
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[df.index[i+251]] = list(df.iloc[i-t+251:i+251]['Adj. Close'])
            Z[f+str(t)] = aux.apply(f,axis=0)  
    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = cross_validation.train_test_split(Xm, ym, test_size = 0.3)
    model =  DecisionTreeRegressor()
    param_grid = dict(criterion = ['mse','mae'],
                      splitter = ['best','random'],          
                      max_features = range(2,15),
                      min_samples_split=range(2,10),
                      max_depth = range(3,15)) 
    grid = GridSearchCV(cv = 3, 
                        estimator = model,
                        n_jobs = -1,
                        param_grid=param_grid)
    grid.fit(Xm,ym)
    model = grid.best_estimator_
    model.fit(Xt,yt)
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

FB
ACC Validate 95.72%
ACC Train    99.10%
GOOGL
ACC Validate 93.63%
ACC Train    99.96%
AAPl
ACC Validate 93.01%
ACC Train    98.16%
AMZN
ACC Validate 98.86%
ACC Train    99.95%
TSLA
ACC Validate 81.01%
ACC Train    84.47%
MSFT
ACC Validate 97.96%
ACC Train    99.76%
AMD
ACC Validate 60.65%
ACC Train    71.95%
INTC
ACC Validate 95.77%
ACC Train    99.02%
T
ACC Validate 77.66%
ACC Train    78.88%
NFLX
ACC Validate 97.69%
ACC Train    98.87%
Promedio Validate  89.19%
Promedio Train     93.01%
CPU times: user 27min 45s, sys: 1min 14s, total: 28min 59s
Wall time: 38min 34s
